read single cell object data

In [ ]:
import json
from livecell_tracker.core.single_cell import SingleCellTrajectoryCollection, SingleCellStatic
from livecell_tracker.track.sort_tracker_utils import (
    track_SORT_bbox_from_scs
)

all_scs_json_path = "./datasets/test_scs_EBSS_starvation/tmp_corrected_scs.json"
all_scs = SingleCellStatic.load_single_cells_json(all_scs_json_path)
sctc = track_SORT_bbox_from_scs(all_scs, raw_imgs=all_scs[0].img_dataset, min_hits=3, max_age=3)

read trained an MMAction model


In [ ]:
import torch

path = r"D:\LiveCellTracker-dev\notebooks\scripts\mmdetection_classify\work_dirs\tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb\epoch_100.pth"
model = torch.load(path)

In [ ]:
from mmengine.config import Config, DictAction
from mmaction.registry import MODELS
config_file = r"D:\LiveCellTracker-dev\notebooks\scripts\mmdetection_classify\tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb.py"
checkpoint_file = r"D:\LiveCellTracker-dev\notebooks\scripts\mmdetection_classify\work_dirs\tsn_imagenet-pretrained-r50_8xb32-1x1x3-100e_kinetics400-rgb\epoch_100.pth"
cfg = Config.fromfile(config_file)

In [ ]:
# model = MODELS.build()
import mmcv
from mmaction.apis import init_recognizer, inference_recognizer
model = init_recognizer(config_file, checkpoint_file, device='cuda:0')

In [ ]:
from mmcv.video.io import VideoReader
import tempfile
import numpy as np

In [ ]:
from livecell_tracker.track.classify_utils import video_frames_and_masks_from_sample

In [ ]:
sample = sctc.get_all_trajectories()[10].get_all_scs()

print("length of sample:", len(sample))
frames, frame_masks = video_frames_and_masks_from_sample(sample)


In [ ]:
sct_frames = [np.array(frame) for frame in frames]

In [ ]:
frames[0].shape

In [ ]:
import cv2
import os
import uuid
# video = r"D:\LiveCellTracker-dev\notebooks\notebook_results\mmaction_train_data\videos\mitosis_1_mask.mp4"
# video = r"D:\LiveCellTracker-dev\notebooks\notebook_results\mmaction_train_data\videos\normal_6.mp4"

tmp_id = uuid.uuid4()
fps = 1
tmp_dir = "./tmp_video"
os.makedirs(tmp_dir, exist_ok=True)
filename = os.path.join(tmp_dir, f"tmp-{tmp_id}.mp4")
input_frames = sct_frames[:10]
with open(filename, "wb+") as tmp_video_file:
    # mmcv.frames2video(frames, tmp_video_file.name, fps=1)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # Create a VideoWriter object to write the frames to the output file
    height, width, channels = input_frames[0].shape
    video_writer = cv2.VideoWriter(tmp_video_file.name, fourcc, fps, (width, height))

    # Write the frames to the output file
    for frame in input_frames:
        video_writer.write(frame)
    video_writer.release()
    tmp_video_file.flush()
    
results = inference_recognizer(model, tmp_video_file.name)
print(tmp_video_file)
results

In [ ]:
results = inference_recognizer(model, r"D:\LiveCellTracker-dev\notebooks\notebook_results\mmaction_train_data\videos\normal_4.mp4")
results